<a href="https://colab.research.google.com/github/dax027/OpenAI-ComplianceAPI/blob/main/ChatGPT_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests, datetime as dt, time, random, re
from typing import List, Dict, Any, Optional, Iterator
import pandas as pd
from google.colab import userdata

# ================= REQUIRED CONFIG =========================
API_KEY = (userdata.get('compliance') or '').strip()
WS_ID   = (userdata.get('WS_ID') or '').strip()
assert API_KEY, "Missing userdata['compliance']"
assert WS_ID,   "Missing userdata['WS_ID']"

BASE = f"https://api.chatgpt.com/v1/compliance/workspaces/{WS_ID}"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json",
    "Content-Type": "application/json",
}

SESSION = requests.Session()
SESSION.headers.update(HEADERS)

# ================== INPUTS =================================
TARGET        = "francisco.rodriguez4@adventhealth.com"   # email or a user-… id
# Format: YYYY, MM, DD, hh, mm
START_UTC = dt.datetime(2025, 9, 9, 0, 0, tzinfo=dt.timezone.utc)
END_UTC   = dt.datetime(2025, 9, 9, 23, 59, tzinfo=dt.timezone.utc)

# If we already know the user id, put it here to skip lookup
USER_ID_OVERRIDE: Optional[str] = 'user-UmhoJIc8cFKiWqQT6rz8RSwi'  # e.g., "user-XXXXXXXXXXXXXXXXXXXXXXXX"

CONV_LIMIT      = 40        # <=500 per spec; drop if 522s persist
USERS_LIMIT     = 200       # default = 200 per spec
MAX_RETRIES     = 6
BASE_BACKOFF    = 0.6
TIMEOUT_CONNECT = 10
TIMEOUT_READ    = 120
PROBE_MIN_SLICE = dt.timedelta(hours=3)

print("Workspace:", WS_ID)
print("Window UTC:", START_UTC.isoformat(), "→", END_UTC.isoformat())


Workspace: 5cce4c27-054b-43f3-b365-4f25fe4619a6
Window UTC: 2025-09-09T00:00:00+00:00 → 2025-09-09T23:59:00+00:00


In [2]:
def _sleep_backoff(attempt: int, base: float = BASE_BACKOFF):
    delay = base * (2 ** attempt) + random.uniform(0, base)
    time.sleep(min(delay, 10))

def _get(url: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    timeout = (TIMEOUT_CONNECT, TIMEOUT_READ)
    for attempt in range(MAX_RETRIES):
        try:
            r = SESSION.get(url, params=params or {}, timeout=timeout)
            if r.status_code == 429:
                print("429 rate-limit — backing off")
                _sleep_backoff(attempt); continue
            if r.status_code in (522, 524) or 500 <= r.status_code < 600:
                print(f"Server/edge error {r.status_code} — retrying")
                _sleep_backoff(attempt); continue
            if r.status_code >= 400:
                print("Compliance API error:", r.status_code, r.text[:300])
                r.raise_for_status()
            return r.json()
        except requests.RequestException as e:
            if attempt == MAX_RETRIES - 1: raise
            _sleep_backoff(attempt)
    raise RuntimeError("Unreachable after retries")

def parse_ts_unix(ts) -> Optional[dt.datetime]:
    if ts is None: return None
    try:
        return dt.datetime.fromtimestamp(float(ts), tz=dt.timezone.utc)
    except Exception:
        return None

def to_unix_seconds(d: dt.datetime) -> int:
    if not d.tzinfo: d = d.replace(tzinfo=dt.timezone.utc)
    return int(d.timestamp())


In [3]:
def _first_page_conversations(start_utc: dt.datetime, limit: int) -> Dict[str, Any]:
    url = f"{BASE}/conversations"
    params = {"since_timestamp": to_unix_seconds(start_utc), "limit": min(limit, 500)}
    return _get(url, params=params)

def _stream_conversations_from_first_page(first_page: Dict[str, Any],
                                          start_utc: dt.datetime,
                                          end_utc: dt.datetime,
                                          limit: int) -> Iterator[Dict[str, Any]]:
    url = f"{BASE}/conversations"
    for conv in first_page.get("data", []): yield conv
    after = first_page.get("last_id") if first_page.get("has_more") else None
    params = {"since_timestamp": to_unix_seconds(start_utc)}
    while after:
        page = _get(url, params={**params, "limit": limit, "after": after})
        for conv in page.get("data", []): yield conv
        after = page.get("last_id") if page.get("has_more") else None

def iter_conversations_adaptive(start_utc: dt.datetime,
                                end_utc: dt.datetime,
                                limit: int,
                                min_slice: dt.timedelta = PROBE_MIN_SLICE) -> Iterator[Dict[str, Any]]:
    """Bisects only on 522/5xx to narrow failing intervals."""
    stack = [(start_utc, end_utc)]
    while stack:
        s, e = stack.pop()
        if s > e: continue
        try:
            first = _first_page_conversations(s, limit)
            yield from _stream_conversations_from_first_page(first, s, e, limit)
        except requests.HTTPError as err:
            code = getattr(err.response, "status_code", None)
            if code and (code in (522, 524) or 500 <= code < 600):
                if (e - s) <= min_slice: raise
                mid = s + (e - s) / 2
                stack.append((s, mid)); stack.append((mid, e))
            else: raise


In [4]:
USER_ID_RE = re.compile(r"^user-[A-Za-z0-9]{8,}$")

def resolve_user_id(identifier: str,
                    start_utc: dt.datetime,
                    end_utc: dt.datetime) -> str:
    # If you’ve pinned an override, use it and print it
    if USER_ID_OVERRIDE:
        print("Using USER_ID_OVERRIDE:", USER_ID_OVERRIDE)
        return USER_ID_OVERRIDE

    ident = (identifier or "").strip()
    if USER_ID_RE.match(ident):
        print("Identifier is already a user_id:", ident)
        print("Resolved user id:", ident)
        return ident

    assert "@" in ident, "Identifier must be an email or a user-… id"
    email = ident.lower()

    print(f"Resolving user ID via /users for {email}")
    url = f"{BASE}/users"
    after = None
    limit = min(max(USERS_LIMIT, 1), 200)
    last_call = 0.0
    pages = 0

    while True:
        # pace to ≤50 req/min
        elapsed = time.time() - last_call
        if elapsed < 1.25:
            time.sleep(1.25 - elapsed)
        params = {"limit": limit}
        if after:
            params["after"] = after
        data = _get(url, params=params)
        last_call = time.time()
        pages += 1

        for u in data.get("data", []):
            if (u.get("email") or "").lower() == email:
                uid = u.get("id")
                print(f"Found via /users on page {pages}.")
                print("Resolved user id:", uid)
                return uid

        if not data.get("has_more"):
            print("End of /users — no match, falling back to probe.")
            break
        after = data.get("last_id")
        if pages % 10 == 0:
            print(f"…/users pages scanned: {pages}")

    print("Probing conversations to infer user id…")
    scanned = 0
    for conv in iter_conversations_adaptive(start_utc, end_utc, limit=min(40, CONV_LIMIT)):
        scanned += 1
        if (conv.get("user_email") or "").lower() == email and conv.get("user_id"):
            uid = conv["user_id"]
            print("Found via conversation.user_id.")
            print("Resolved user id:", uid)
            return uid
        for m in ((conv.get("messages") or {}).get("data") or []):
            a = m.get("author") or {}
            if (a.get("email") or "").lower() == email and a.get("id"):
                uid = a["id"]
                print("Found via message.author.id.")
                print("Resolved user id:", uid)
                return uid
        if scanned % 20 == 0:
            print(f"…probed {scanned} conversations")

    raise RuntimeError(f"Could not resolve user_id for {email}")


In [5]:
def pin_user_id_override(uid: str):
    global USER_ID_OVERRIDE
    USER_ID_OVERRIDE = uid
    print("USER_ID_OVERRIDE pinned:", USER_ID_OVERRIDE)

# Example after resolving once:
# uid = resolve_user_id(TARGET, START_UTC, END_UTC)
# pin_user_id_override(uid)


In [6]:
def iter_conversations_for_user_window(user_id: str,
                                       start_utc: dt.datetime,
                                       end_utc: dt.datetime,
                                       limit: int = CONV_LIMIT) -> Iterator[Dict[str, Any]]:
    """
    - First page: since_timestamp + users + file_format=id (lighter payload)
    - Next pages: after + users + file_format=id
    - On 5xx/522: backoff and shrink current limit to reduce payload
    - Gentle pacing between pages
    """
    url = f"{BASE}/conversations"
    limit_cur = max(10, min(int(limit), 500))
    base_params = {"users": user_id, "file_format": "id"}

    def _page(params, max_attempts=6):
        nonlocal limit_cur
        for attempt in range(max_attempts):
            try:
                return _get(url, params=params)
            except requests.HTTPError as err:
                code = getattr(err.response, "status_code", None)
                if code and (code in (522, 524) or 500 <= code < 600):
                    if limit_cur > 20:
                        limit_cur = max(20, limit_cur // 2)
                        print(f"5xx on conversations — shrinking limit to {limit_cur}")
                    _sleep_backoff(attempt)
                    continue
                raise

    # First page (since_timestamp)
    first_params = {**base_params, "since_timestamp": to_unix_seconds(start_utc), "limit": limit_cur}
    first = _page(first_params)

    for conv in (first.get("data") or []):
        la = parse_ts_unix(conv.get("last_active_at"))
        if la and la > end_utc:
            continue
        yield conv

    after = first.get("last_id") if first.get("has_more") else None

    # Follow-on pages (after)
    while after:
        time.sleep(0.3)  # gentle pacing
        page = _page({**base_params, "after": after, "limit": limit_cur})
        for conv in (page.get("data") or []):
            la = parse_ts_unix(conv.get("last_active_at"))
            if la and la > end_utc:
                continue
            yield conv
        after = page.get("last_id") if page.get("has_more") else None


def flatten_messages_from_conversation(conv: Dict[str, Any],
                                       start_utc: dt.datetime,
                                       end_utc: dt.datetime) -> List[Dict[str, Any]]:
    out = []
    for m in ((conv.get("messages") or {}).get("data") or []):
        ts = parse_ts_unix(m.get("created_at"))
        if not ts or ts < start_utc or ts > end_utc:
            continue
        a = m.get("author") or {}; c = m.get("content") or {}
        files = ((m.get("files") or {}).get("data") or [])
        f_summary = "; ".join(f"{f.get('name','(no name)')}|{f.get('id','')}" for f in files)
        out.append({
            "conversation_id": conv.get("id"),
            "conversation_title": conv.get("title"),
            "message_id": m.get("id"),
            "message_created_at": ts.isoformat(),
            "author_role": a.get("role"),
            "author_id": a.get("id"),
            "author_email": a.get("email"),
            "content_type": c.get("type"),
            "content_value": c.get("value"),
            "file_summary": f_summary,
        })
    return out


def collect_user_messages_window(identifier: str,
                                 start_utc: dt.datetime,
                                 end_utc: dt.datetime) -> List[Dict[str, Any]]:
    user_id = resolve_user_id(identifier, start_utc, end_utc)
    all_rows = []
    scanned = 0
    for conv in iter_conversations_for_user_window(user_id, start_utc, end_utc, limit=CONV_LIMIT):
        scanned += 1
        rows = flatten_messages_from_conversation(conv, start_utc, end_utc)
        all_rows.extend(rows)
        if scanned % 25 == 0:
            print(f"Processed {scanned} conversations; {len(all_rows)} messages so far.")
    print(f"Conversations considered: {scanned}")
    print(f"Messages in window: {len(all_rows)}")
    return all_rows


In [10]:
# Cell 7: list conversations in Eastern Time, with file counts (no auto-export)
import pytz
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 0)

eastern = pytz.timezone("US/Eastern")

uid = resolve_user_id(TARGET, START_UTC, END_UTC)
print("Target user id:", uid)

CONV_CACHE: Dict[str, Dict[str, Any]] = {}
rows = []

for i, conv in enumerate(iter_conversations_for_user_window(uid, START_UTC, END_UTC, limit=CONV_LIMIT), 1):
    conv_id = conv.get("id")
    title = conv.get("title")

    # timestamps
    last_active_utc = parse_ts_unix(conv.get("last_active_at"))
    last_active_et = (
        pd.to_datetime(last_active_utc, utc=True).tz_convert(eastern)
        if last_active_utc else None
    )

    # message + file counts
    msgs = ((conv.get("messages") or {}).get("data")) or []
    message_count = len(msgs)

    # Sum file references across all messages
    total_files = 0
    sample_file_ids = []
    for m in msgs:
        files = ((m.get("files") or {}).get("data")) or []
        total_files += len(files)
        # preview a few file ids for situational awareness
        for f in files:
            fid = f.get("id")
            if fid and len(sample_file_ids) < 5:
                sample_file_ids.append(fid)

    if conv_id:
        CONV_CACHE[conv_id] = conv
        rows.append({
            "conversation_id": conv_id,
            "conversation_title": title,
            "last_active_et": last_active_et.isoformat() if last_active_et else None,
            "message_count": message_count,
            "file_count": total_files,
            "sample_file_ids": ", ".join(sample_file_ids) if sample_file_ids else None,
        })

    if i % 25 == 0:
        print(f"…indexed {i} conversations so far")

conv_df = pd.DataFrame(rows)
if not conv_df.empty:
    conv_df["last_active_et_parsed"] = pd.to_datetime(conv_df["last_active_et"])
    conv_df = conv_df.sort_values("last_active_et_parsed", ascending=False).drop(columns=["last_active_et_parsed"])
    display(conv_df.head(100))
else:
    print("No conversations found in the selected window.")

def export_conversations_index(path="/content/conversations_index.csv"):
    """Export conversation index manually."""
    if 'conv_df' not in globals() or conv_df.empty:
        print("Nothing to export.")
        return
    conv_df.to_csv(path, index=False)
    print("Exported to:", path)


Using USER_ID_OVERRIDE: user-UmhoJIc8cFKiWqQT6rz8RSwi
Target user id: user-UmhoJIc8cFKiWqQT6rz8RSwi


,conversation_id,conversation_title,last_active_et,message_count,file_count,sample_file_ids
1,68b2010f-4470-8330-8451-a6d66df0ff66,Greeting conversation,2025-09-09T16:12:52.611337-04:00,39,6,"file-1YQZUW1KHTwWzN7E8rbVer, file-YX11Ht6JTKnFdFeRfxj8Qg, file-XCMYLaqbJZaMgy2JVFC4WE, file-VUciHnTy49zQ1KjgVWV4oF, file-VuYRoRVxuEBFZKZjX95P4x"
0,68ac7ae9-f67c-8332-90fa-2d5d711de754,Export PrintService logs,2025-09-09T09:11:27.443272-04:00,79,0,None


In [11]:
# Cell 8: drill into one conversation, full message text in ET + file details (no auto-export)
import pytz

def messages_df_for(conversation_id: str,
                    start_utc: dt.datetime,
                    end_utc: dt.datetime,
                    tz: str = "US/Eastern") -> pd.DataFrame:
    """Build DataFrame of messages for one conversation from CONV_CACHE, including file columns."""
    if conversation_id not in CONV_CACHE:
        print("Conversation not in cache:", conversation_id)
        return pd.DataFrame()

    conv = CONV_CACHE[conversation_id]
    msgs = ((conv.get("messages") or {}).get("data")) or []

    rows = []
    for m in msgs:
        ts_utc = parse_ts_unix(m.get("created_at"))
        if not ts_utc or ts_utc < start_utc or ts_utc > end_utc:
            continue

        author = m.get("author") or {}
        content = m.get("content") or {}

        files = ((m.get("files") or {}).get("data") or [])
        file_ids = [f.get("id") for f in files if f.get("id")]
        # Some schemas include name; if missing, we’ll show None gracefully
        file_names = [f.get("name") for f in files if f.get("name")]

        rows.append({
            "message_created_at": ts_utc.isoformat(),  # UTC ISO
            "author_role": author.get("role"),
            "author_email": author.get("email"),
            "content_value": content.get("value"),     # full text; no truncation due to pandas option
            "message_id": m.get("id"),
            "file_count": len(files),
            "file_ids": ", ".join(file_ids) if file_ids else None,
            "file_names": ", ".join(file_names) if file_names else None,
        })

    if not rows:
        print("No messages for that conversation in window.")
        return pd.DataFrame()

    dfm = pd.DataFrame(rows).sort_values("message_created_at")
    tzinfo = pytz.timezone(tz)
    dfm["message_created_at_et"] = (
        pd.to_datetime(dfm["message_created_at"], utc=True).dt.tz_convert(tzinfo)
    )
    return dfm

# Example: pick first conversation
if 'conv_df' in globals() and not conv_df.empty:
    conv_id = conv_df.iloc[0]["conversation_id"]
    print("Selected conversation:", conv_id)
    df_msgs = messages_df_for(conv_id, START_UTC, END_UTC)
    if not df_msgs.empty:
        display(df_msgs[[
            "message_created_at_et","author_role","author_email","content_value",
            "file_count","file_names","file_ids","message_id"
        ]])
else:
    print("Run Cell 7 first to build conv_df.")

def export_messages_for(conversation_id: str, path="/content/messages.csv"):
    """Export messages manually if desired."""
    dfm = messages_df_for(conversation_id, START_UTC, END_UTC)
    if dfm.empty:
        print("No messages to export.")
        return
    dfm.to_csv(path, index=False)
    print("Exported messages to:", path)


Selected conversation: 68b2010f-4470-8330-8451-a6d66df0ff66


,message_created_at_et,author_role,author_email,content_value,file_count,file_names,file_ids,message_id
0,2025-09-09 14:33:46.577000-04:00,user,None,New Ask,0,None,None,68f972ac-908d-4e6f-8d76-8e85596b930a
1,2025-09-09 14:33:49.116652-04:00,system,None,None,0,None,None,1a3044ef-b4a3-4f27-8b00-ebbbac7f5101
2,2025-09-09 14:33:49.179349-04:00,assistant,None,Got it 👍 — what’s your new ask?,0,None,None,4d7f7c36-85bc-4df1-9929-bf2d0ff438ab
3,2025-09-09 14:37:26.248000-04:00,user,None,"Parse the data and create a excel file that is going to show The PrintServers with the number of Printers, SizeMB and Pages printer per date",6,"FHOVPRNF001 Parsed_Data_20250909.csv, FHOVPRNE001 Parsed_Data_20250909.csv, FHOVPRND001 Parsed_Data_20250909.csv, FHOVPRNC001 Parsed_Data_20250909.csv, FHOVPRNB001 Parsed_Data_20250909.csv, FHOVPRNA001 Parsed_Data_20250909.csv","file-1YQZUW1KHTwWzN7E8rbVer, file-YX11Ht6JTKnFdFeRfxj8Qg, file-XCMYLaqbJZaMgy2JVFC4WE, file-VUciHnTy49zQ1KjgVWV4oF, file-VuYRoRVxuEBFZKZjX95P4x, file-Wzh28UZqpzwDHegtgLbxjw",e10e7397-4a61-4c68-8d19-9a4e8e37b0b8
4,2025-09-09 14:37:28.230157-04:00,system,None,None,0,None,None,18731f11-b470-493c-9d97-2a74f1a64e21
5,2025-09-09 14:37:28.244915-04:00,assistant,None,"import pandas as pd\r\nimport glob\r\nimport os\r\n\r\n# File paths from uploaded files\r\nfiles = [\r\n ""/mnt/data/FHOVPRNF001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNE001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRND001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNC001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNB001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNA001 Parsed_Data_20250909.csv"",\r\n]\r\n\r\n# Load all CSVs into a single DataFrame with PrintServer identification\r\ndf_list = []\r\nfor f in files:\r\n try:\r\n temp = pd.read_csv(f)\r\n temp[""PrintServer""] = os.path.basename(f).split()[0] # Extract server name\r\n df_list.append(temp)\r\n except Exception as e:\r\n print(f""Error reading {f}: {e}"")\r\n\r\ndf = pd.concat(df_list, ignore_index=True)\r\n\r\n# Inspect structure\r\ndf.head()",0,None,None,53c42575-de9f-45f1-b08e-62e02bf56b55
6,2025-09-09 14:37:42.307995-04:00,tool,None,Code execution state reset.,0,None,None,6bab5cdd-e406-49f1-8d74-62ef06fdf78c
7,2025-09-09 14:37:42.506574-04:00,assistant,None,"import pandas as pd\r\nimport os\r\n\r\n# File paths again after reset\r\nfiles = [\r\n ""/mnt/data/FHOVPRNF001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNE001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRND001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNC001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNB001 Parsed_Data_20250909.csv"",\r\n ""/mnt/data/FHOVPRNA001 Parsed_Data_20250909.csv"",\r\n]\r\n\r\n# Load all CSVs into a single DataFrame with PrintServer identification\r\ndf_list = []\r\nfor f in files:\r\n try:\r\n temp = pd.read_csv(f)\r\n temp[""PrintServer""] = os.path.basename(f).split()[0] # Extract server name\r\n df_list.append(temp)\r\n except Exception as e:\r\n print(f""Error reading {f}: {e}"")\r\n\r\ndf = pd.concat(df_list, ignore_index=True)\r\n\r\n# Show first rows and columns to understand structure\r\ndf.head(), df.columns",0,None,None,34fb6727-f2e0-499d-84d1-bce66561bb67
8,2025-09-09 14:37:48.416167-04:00,tool,None,"( TimeCreated User Machine \\n 0 9/9/2025 11:41:43 AM DPOC63 FMGLD1THD74 \n 1 9/9/2025 11:41:32 AM KGE87C FMGD51KCPR3 \n 2 9/9/2025 11:41:22 AM NLWF4E ALTDGRG1PK3 \n 3 9/9/2025 11:41:11 AM AKC365 FMGD9D9LG04 \n 4 9/9/2025 11:41:04 AM BMAAB0 FMGDAHBXZMMH3 \n \n DocumentName Printer PrintServer \\n 0 Hybrent OPA4 FHOVPRNF001 \n 1 about_blank NGF4 FHOVPRNF001 \n 2 request.pdf IWF2 FHOVPRNF001 \n 3 Insurance Card - MRN_ 506269459 - Pt. Name_ BR... NF82 FHOVPRNF001 \n 4 EpicPDFSpooler_24BBCE7CA96E4BFB8E6A3303E2E42F17 OMJZ FHOVPRNF001 \n \n PrinterPort SizeMB PagesPrinted \n 0 10.100.39.52 0.86 1 \n 1 10.28.244.217 0.38 2 \n 2 10.16.246.255 31.09 2 \n 3 10.48.247.51 0.21 1 \n 4 10.89.8.29 0.28 1 ,\n Index(['TimeCreated', 

In [12]:
# Cell 9: show a window of messages around a given message_id, including file info
def convo_thread_window(df_msgs: pd.DataFrame,
                        anchor_message_id: str,
                        before: int = 3,
                        after: int = 3) -> pd.DataFrame:
    if df_msgs is None or df_msgs.empty:
        print("No df_msgs loaded.")
        return pd.DataFrame()
    df_sorted = df_msgs.sort_values("message_created_at").reset_index(drop=True)
    idx = df_sorted.index[df_sorted["message_id"] == anchor_message_id]
    if len(idx) == 0:
        print("Message not found:", anchor_message_id)
        return pd.DataFrame()
    i = int(idx[0])
    start_i, end_i = max(0, i - before), min(len(df_sorted), i + after + 1)
    window = df_sorted.iloc[start_i:end_i].copy()
    display(window[[
        "message_created_at_et","author_role","author_email","content_value",
        "file_count","file_names","file_ids","message_id"
    ]])
    return window

# Example:
# anchor_id = df_msgs.iloc[5]["message_id"]
# convo_thread_window(df_msgs, anchor_id, before=5, after=5)
